# Global Uncertainty Analysis: Polynomial Chaos Expansion (PCE) for Chemical Reaction Systems


This ipython notebook uses MUQ as a basis for adaptive Polynomial Chaos Expansions to perform global uncertainty analysis for chemical reaction systems.  This ipython notebook details a workflow using RMG, Cantera, and MUQ codes.

Muq binary only works on linux systems, please also add the ~/anaconda/envs/your_env/lib folder to your $PYTHONPATH to import muq smoothly.

In [ ]:
from rmgpy.tools.canteraModel import Cantera, getRMGSpeciesFromUserSpecies
from rmgpy.species import Species
from rmgpy.chemkin import loadChemkinFile
from rmgpy.tools.muq import ReactorPCEFactory
from rmgpy.tools.uncertainty import Uncertainty

Load chemical kinetic mechanism from RMG chemkin file and dictionary.

In [ ]:
# The paths for the chemkin file and its species dictionary
chemkinFile = 'uncertainty/chem_annotated.inp'
dictFile = 'uncertainty/species_dictionary.txt'

For uncorrelated uncertainty studies, we can simply create a cantera job for the model using species and reactions lists from the `loadChemkinFile` function.  Alternatively the `Uncertainty` class's `loadModel` function can be used.

In [ ]:
outputDir = 'uncertaintyUncorrelated'
speciesList, reactionList = loadChemkinFile(chemkinFile, dictFile)

# Declare some species that we want to use as initial conditions or in our uncertainty analysis
PDD = Species().fromSMILES("CCCCCCCCCCCCc1ccccc1")
C11ene=Species().fromSMILES("CCCCCCCCCC=C")
ETHBENZ=Species().fromSMILES("CCc1ccccc1")

# Map the species to their respective objects in the speciesList and reactionList
mapping = getRMGSpeciesFromUserSpecies([PDD,C11ene,ETHBENZ], speciesList)

reactorTypeList = ['IdealGasConstPressureTemperatureReactor']
molFracList = [{mapping[PDD]: 1.0}]
Tlist = ([623],'K')
Plist = ([350],'bar')
reactionTimeList = ([72], 'h')

# Create the cantera model
job = Cantera(speciesList=speciesList, reactionList=reactionList, outputDirectory=outputDir)
# Load the cantera model based on the RMG reactions and species
job.loadModel()
# Generate the conditions based on the settings we declared earlier
job.generateConditions(reactorTypeList, reactionTimeList, molFracList, Tlist, Plist)

Let's create a second cantera model and output folder to use for the correlated uncertainty analysis.  Be careful to use NEW species and reaction objects, since running the two cantera models through the global uncertainty analysis module at the same time inside a single python script may cause collision issues (the ReactorPCEFactory class actively manipulates species and reaction object data).  For an uncorrelated analysis, we must use the `Uncertainty` class, because we need to pass in the partial input uncertainty dictionaries: `Uncertainty.kineticInputUncertainties` and `Uncertainty.thermoInputUncertinaties` into the global analysis uncertainty class.

In [ ]:
outputDir = 'uncertaintyCorrelated'

uncertainty = Uncertainty(outputDirectory='testUncertainty')
uncertainty.loadModel(chemkinFile, dictFile)

Use the Uncertainty class to assign correlated uncertainties, which we will propagate.  This requires an additional step of loading the RMG database and extracting the original parameter sources (i.e. rate rules and thermo)

In [ ]:
uncertainty.loadDatabase()
uncertainty.extractSourcesFromModel()
# Assign correlated parameter uncertainties 
uncertainty.assignParameterUncertainties(correlated=True)

Again create a Cantera model object that stores the reaction conditions.

In [ ]:
mappingCorrelated = getRMGSpeciesFromUserSpecies([PDD,C11ene,ETHBENZ], uncertainty.speciesList)

reactorTypeList = ['IdealGasConstPressureTemperatureReactor']
molFracList = [{mappingCorrelated[PDD]: 1.0}]
Tlist = ([623],'K')
Plist = ([350],'bar')
reactionTimeList = ([72], 'h')

jobCorrelated = Cantera(speciesList=uncertainty.speciesList, reactionList=uncertainty.reactionList, outputDirectory=outputDir)
# Load the cantera model based on the RMG reactions and species
jobCorrelated.loadModel()
# Generate the conditions based on the settings we declared earlier
jobCorrelated.generateConditions(reactorTypeList, reactionTimeList, molFracList, Tlist, Plist)


Input a set of kinetic $(k)$ and thermo $(G)$ parameters to be propagated and their uncertainties $(d\ln(k), dG)$ into the `ReactorPCEFactory` class.  These kinetic and thermo parameters should typically be pre-screened from local uncertainty analysis to narrow down to the most influential parameters.  
Each parameter's uncertainty is considered to be a uniform uncertainty interval where unit random variables $\ln(k)_{rv}$ and $G_{rv}$ are mapped to the user-assigned parameter uncertainties.

$\ln(k)_{rv} \sim U(-1, 1) \rightarrow ln(k) \sim U(-d\ln(k), d\ln(k))$

$G_{rv} \sim U(-1, 1) \rightarrow G \sim U(-dG, dG)$


Polynomial chaos expansions (PCE) are contructed for the desired outputs of interest (species mole fractions).

In the case of correlated parameters, we will need to indicate them in the `ReactorPCEFactory` object, and as well as provide the dictionaries of assigned partial uncertainties from the `Uncertainty` class.

In [ ]:
# Create ReactorPCEFactory global uncertainty analysis object for the uncorrelated case

reactorPCEFactory = ReactorPCEFactory(cantera=job,
                            outputSpeciesList=[mapping[PDD], mapping[C11ene]],
                            kParams=[796, 801], # [Styrene+Decyl=Rad1, c10ene + ebzyl = rad4]
                                    # A list of indices corresponding to the uncertain reactions
                            kUncertainty = [1.414, 1.414],   
                                      # a list of dlnk corresponding to uncertainties of kParams
                            gParams = [4,5],  # A list of indices corresponding to the uncertain thermo   [PDD, Toluene]
                            gUncertainty = [2.0, 1.0],
                                      # a list of values corresponding to dG
                            correlated = False 
                            )

reactorPCEFactoryCorrelated = ReactorPCEFactory(cantera=jobCorrelated,
                            outputSpeciesList=[mappingCorrelated[PDD], mappingCorrelated[C11ene]],
                            kParams=['R_Addition_MultipleBond Cds-HH_Cds-CsH;CsJ-CsHH', 
                                    'Estimation STYRENE(3)+DECYL(56)=RAD1(14)'],   
                                      # labels for the correlated kinetics parameters
                            kUncertainty = uncertainty.kineticInputUncertainties,   
                                      # a list of dictionaries that gives the reaction's partial uncertainties
                                      # with respect to the string labels of the kinetic correlated parameters, i.e. 'H_Abstraction CHO/Oa'
                            gParams = ['Group(group) Cs-CsCsHH', 'Library TOLUENE(2)'],  # labels for the correlated thermo parameters
                            gUncertainty = uncertainty.thermoInputUncertainties,
                                      # a list of dictionaries that gives the species partial uncertainties
                                      # with respect to the string labels of the correlated thermo parameters, i.e. 'Group(ring) cyclohexane'
                            correlated = True   
                            )



Begin generating the PCEs adaptively based a runtime.

There are actually three methods for generating PCEs. See the `ReactorPCEFactory.generatePCE` function for more details.

- Option 1: Adaptive for a pre-specified amount of time
- Option 2: Adaptively construct PCE to error tolerance
- Option 3: Used a fixed order, and (optionally) adapt later.  

In [ ]:
reactorPCEFactory.generatePCE(runTime=60)  # runtime of 60 seconds.

Let's compare the outputs for a test point using the real model versus using the PCE approximation.
Evaluate the desired output mole fractions based on a set of inputs `ins = [[` $\ln(k)_{rv}$ `], [` $G_{rv}$ `]]` which contains the 
random unit uniform variables attributed to the uncertain kinetics and free energy parameters, respectively.

In [ ]:
trueTestPointOutput, pceTestPointOutput = reactorPCEFactory.compareOutput([0.5,0.2,0.1,-.7])

Obtain the results: the species mole fraction mean and variance computed from the PCE, as well as the global sensitivity indices.

In [ ]:
mean, variance, covariance, mainSens, totalSens = reactorPCEFactory.analyzeResults()

Do the same analysis for the correlated `reactorPCEFactory`

In [ ]:
reactorPCEFactoryCorrelated.generatePCE(runTime=60)  # runtime of 60 seconds.

In [ ]:
trueTestPointOutput, pceTestPointOutput = reactorPCEFactoryCorrelated.compareOutput([0.5,0.2,0.1,-.7])

In [ ]:
mean, variance, covariance, mainSens, totalSens = reactorPCEFactoryCorrelated.analyzeResults()